In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
! pip install accelerate -U

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

In [2]:
pip install --upgrade datasets

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [3]:
from datasets import load_dataset, concatenate_datasets

# insert your code here
dataset = load_dataset("yelp_polarity")
train_dataset = dataset['train']
test_dataset = dataset['test']

# Display some information about the datasets
print(f"Training set size: {len(train_dataset)}")
print(f"Testing set size: {len(test_dataset)}")

# Access and print the first sample from the training set
print("First training example:", train_dataset[0])
print("First testing example:", test_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Training set size: 560000
Testing set size: 38000
First training example: {'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.", 'label': 0}
First testing example: {'text': "Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experi

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [4]:
# insert your code here
import random

# Select 150 elements from the train set with label = 0 and 150 elements with label = 1
train_dataset_0 = train_dataset.filter(lambda example: example["label"]==0).select([i for i in range(0,150)])
train_dataset_1 = train_dataset.filter(lambda example: example["label"]==1).select([i for i in range(0,150)])

# Concatenate the two sub-datasets
train_dataset_ordered = concatenate_datasets([train_dataset_0,train_dataset_1])

# Shuffle it in order to acquire the final train_dataset with 300 elements
train_dataset = train_dataset_ordered.shuffle(seed=random. randint(10, 100))

test_dataset_0 = test_dataset.filter(lambda example: example["label"]==0).select([i for i in range(0,150)])
test_dataset_1 = test_dataset.filter(lambda example: example["label"]==1).select([i for i in range(0,150)])
test_dataset_ordered = concatenate_datasets([test_dataset_0,test_dataset_1])
test_dataset = test_dataset_ordered.shuffle(seed=17)

print("Train Dataset")
print(train_dataset)
print("\nTest Dataset")
print(test_dataset)


Train Dataset
Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

Test Dataset
Dataset({
    features: ['text', 'label'],
    num_rows: 300
})


In [5]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [6]:
# insert your code here
'''
Sequence classification is a type of natural language processing (NLP) task where a model is assigned to categorize an entire sequence of text into one or more classes >
here we have label 0, 1 corresponding to negative or positive comment
'''
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base") #RoBERTa model for sequence classification tasks, its a transformer
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [7]:
'''
text is first tokenized into words or subword units, converted into numerical IDs
'''
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [8]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [9]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.



---



Lr = 0.0001

Epochs = 3

Batch size = 16

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_scheduler

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16, learning_rate=0.0001, num_train_epochs=3, weight_decay=0.01, logging_steps=100)

# insert your code here
#we define the defaults, could skip

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001) #handle weight decay differently than adam for better performance

total_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs

'''
Functions of a Learning Rate Scheduler:
Control Learning Rate Dynamically
Avoid Local Minima and Plateaus
Fine-tuning Learning Process: Starting with a higher learning rate can help in exploring the solution space quickly, and gradually lowering the rate can help in fine-tuning the adjustments to model weights, leading to a more precise convergence
'''

#linear scheduler, which linearly decreases the learning rate from the initial lr set in the optimizer to zero,
#over the course of training, num_warmup_steps specifies the number of steps to linearly increase the learning rate
#from zero to the initial learning rate (as a form of warmup strategy)
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(total_steps * 0.1),  # during the first 10% of training the learning rate warms up from 0 to 0.0001
   num_training_steps=total_steps
)

# etc

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer,scheduler)
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.335560,0.886667
2,No log,0.227580,0.916667


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.335560,0.886667
2,No log,0.227580,0.916667
3,No log,0.321270,0.916667




---

Lr = 0.0001

Epochs = 3

Batch size = 8

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_scheduler

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8, learning_rate=0.0001, num_train_epochs=3, weight_decay=0.01, logging_steps=100)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

total_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(total_steps * 0.1),  # Warmup 10% of the total steps
   num_training_steps=total_steps
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer,scheduler)
)

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.375364,0.896667
2,No log,0.607532,0.846667
3,0.454400,0.279822,0.943333




---

Lr = 0.001

Epochs = 3

Batch size = 8


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_scheduler

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8, learning_rate=0.001, num_train_epochs=3, weight_decay=0.01, logging_steps=100)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

total_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(total_steps * 0.1),  # Warmup 10% of the total steps
   num_training_steps=total_steps
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer,scheduler)
)

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.706940,0.500000
2,No log,0.699485,0.500000
3,0.789000,0.695008,0.500000




---



Lr = 0.002

Epochs = 3

Batch size = 8

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_scheduler
#des an douleve kai alaxe to lr
args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8, learning_rate=0.002, num_train_epochs=3, weight_decay=0.01, logging_steps=100)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.002)

total_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(total_steps * 0.1),  # Warmup 10% of the total steps
   num_training_steps=total_steps
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer,scheduler)
)

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.696860,0.500000
2,No log,0.743571,0.500000


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.696860,0.500000
2,No log,0.743571,0.500000
3,0.806000,0.695985,0.500000




---



Lr = 0.00005

Epochs = 5

Batch Size = 8

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_scheduler

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8, learning_rate=0.00005, num_train_epochs=5, weight_decay=0.01, logging_steps=100)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005)

total_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(total_steps * 0.1),  # Warmup 10% of the total steps
   num_training_steps=total_steps
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer,scheduler)
)

In [11]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.639827,0.510000
2,No log,0.429854,0.793333
3,0.521800,0.353655,0.926667
4,0.521800,0.364880,0.923333
5,0.521800,0.340008,0.936667




---


Lr = 0.0001

Epochs = 5

Batch size = 8

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_scheduler

args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=8, learning_rate=0.0001, num_train_epochs=5, weight_decay=0.01, logging_steps=100)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

total_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(total_steps * 0.1),  # Warmup 10% of the total steps
   num_training_steps=total_steps
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer,scheduler)
)

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.095716,0.780000
2,No log,0.472321,0.903333


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.095716,0.780000
2,No log,0.472321,0.903333
3,0.412900,0.526117,0.896667
4,0.412900,0.703516,0.893333
5,0.412900,0.718898,0.900000


Τα αποτελέσματα συνοψίζονται στο παρακάτω πινακάκι:

|          | Learning rate | Epochs | Batch size | Accuracy |
|----------|----------|----------|----------|----------|
| **Πείραμα 1**| 0.0001  | 3  | 16  | 0.92  |
| **Πείραμα 2**| 0.0001  | 3  | 8  | 0.943  |
| **Πείραμα 3**| 0.0001  | 5  | 8  | 0.9  |
| **Πείραμα 4**| 0.001  | 3  | 8  | 0.5  |
| **Πείραμα 5**| 0.002  | 3  | 8  | 0.5  |
| **Πείραμα 6**| 0.00005  | 5  | 8  | 0.947  |



---



Χρησιμοποιώ σχετικά μικρό πλήθος εποχών, αφού το μοντέλο είναι προεκπαιδευμένο και χρησιμοποιώ μικρό batch size, για να προλάβει να προσαρμοστεί στα δεδομένα μου.
Προφανώς η μείωση του batch size και η αύξηση των εποχών δίνει καλύτερη ακρίβεια, καθώς το μοντέλο προσαρμόζεται καλύτερα στα δεδομένα μου (η ασυνέπεια στο 3ο πείραμα οφείλεται σε underfitting). Παρατηρώ ακόμη πως αν αυξήσω λίγο το learning rate το μοντέλο δεν μπορεί να εκπαιδευτεί, διότι προκαλεί τον optimiser να υπερβαίνει τα βέλτιστα σημεία κατά τη διάρκεια της εκπαίδευσης.

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
import random
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("piqa")

train_dataset = dataset["train"]
test_dataset = dataset['validation'] #test is -1 everywhere for some reason

test_dataset = test_dataset.shuffle(seed=random. randint(10, 100))
# Select 50 elements set with label = 0 and 50 elements with label = 1
test_dataset_0 = test_dataset.filter(lambda example: example["label"] == 0).select([i for i in range(0, 50)])
test_dataset_1 = test_dataset.filter(lambda example: example["label"] == 1).select([i for i in range(0, 50)])
# Concatenate the two sub-datasets
test_dataset_ordered = concatenate_datasets([test_dataset_0, test_dataset_1])
# Shuffle the final dataset
test_dataset = test_dataset_ordered.shuffle(seed=random. randint(10, 100))

train_dataset = train_dataset.shuffle(seed=random. randint(10, 100))
train_dataset_0 = train_dataset.filter(lambda example: example["label"] == 0).select([i for i in range(0, 50)])
train_dataset_1 = train_dataset.filter(lambda example: example["label"] == 1).select([i for i in range(0, 50)])
train_dataset_ordered = concatenate_datasets([train_dataset_0, train_dataset_1])
train_dataset = train_dataset_ordered.shuffle(seed=random. randint(10, 100))

# Display some information about the datasets
print(f"Training set :", train_dataset)
print(f"Testing set :", test_dataset)

Filter:   0%|          | 0/1838 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1838 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16113 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16113 [00:00<?, ? examples/s]

Training set : Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})
Testing set : Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})


Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
# insert your code here (models)
from transformers import AutoTokenizer, BertForMultipleChoice, DistilBertForMultipleChoice, AlbertForMultipleChoice,  BigBirdForMultipleChoice, CamembertForMultipleChoice

# Bert Model
model_name_bert = "bert-base-uncased"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
model_bert = BertForMultipleChoice.from_pretrained(model_name_bert)

# Distilbert Model
model_name_distil = "distilbert-base-cased"
tokenizer_distil = AutoTokenizer.from_pretrained(model_name_distil)
model_distil = DistilBertForMultipleChoice.from_pretrained(model_name_distil)

# Albert Model
model_name_albert = "albert-base-v2"
tokenizer_albert = AutoTokenizer.from_pretrained(model_name_albert)
model_albert = AlbertForMultipleChoice.from_pretrained(model_name_albert)

# Big Bird Model
model_name_bigbird = "google/bigbird-roberta-base"
tokenizer_bigbird = AutoTokenizer.from_pretrained(model_name_bigbird)
model_bigbird = BigBirdForMultipleChoice.from_pretrained(model_name_bigbird)

# Camembert Model
model_name_camem = "camembert-base"
tokenizer_camem = AutoTokenizer.from_pretrained(model_name_camem)
model_camem = CamembertForMultipleChoice.from_pretrained(model_name_camem)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForMultipleChoice were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# insert your code here (function for ending prediction)
def sentence_complete(model, tokenizer, sentence, opt1, opt2):

  # Tokenize inputs
  inputs = tokenizer(
      [sentence] * 2, # repeat the sentence twice, the input batch will contain two sequences, tokenized such that they can be processed as a batch by the model, with padding and truncation applied as needed
      [opt1,opt2],
      padding=True, #ensure that the tokenized inputs are padded to the same length, which is important for batch processing
      truncation=True,
      return_tensors="pt"
  )

  '''
  input_ids are an array of integers representing the text input after tokenization,
  each token is mapped to an integer ID using a predefined dictionary (vocabulary) specific to the model

  attention_mask is an array (of the same shape as input_ids) that contains 1s and 0s, it indicates to the model
  which tokens should be attended to, and which should not,
  useful in padding: it and should not influence the model’s understanding of the actual content
  '''
  input_ids = inputs["input_ids"].unsqueeze(0) #extract from the tokenized inputs
  attention_mask = inputs["attention_mask"].unsqueeze(0)

  model.eval()
  # Forward pass
  with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask) #the used models are designed to handle multiple inputs at once, both sentence-option pairs are evaluated simultaneously, but within the same forward pass because they are part of one batch

  logits = outputs.logits #a logit for each input sequence, each logit in this tensor represents the model's confidence in each respective option being the correct or more appropriate completion of the initial sentence
  predicted_label = logits.argmax(dim=1)
  return predicted_label[0]

In [ ]:
import numpy as np

def compute_accuracy(model,tokenizer):
  pred = np.array([sentence_complete(model, tokenizer, goal, sol1, sol2) for goal,
                   sol1, sol2 in zip(test_dataset['goal'], test_dataset['sol1'],
                                     test_dataset['sol2'])])
  true_labels = np.array(test_dataset['label'])
  accuracy = (pred == true_labels).mean()
  print("Accuracy:", accuracy)

print("=================== Bert Model ===================")
compute_accuracy(model_bert,tokenizer_bert)
print("=================== DistilBert Model ===================")
compute_accuracy(model_distil,tokenizer_distil)
print("=================== Albert Model ===================")
compute_accuracy(model_albert,tokenizer_albert)
print("=================== BigBird Model ===================")
compute_accuracy(model_bigbird,tokenizer_bigbird)
print("=================== Camembert Model ===================")
compute_accuracy(model_camem,tokenizer_camem)

=================== Bert Model ===================
Accuracy: 0.5
=================== DistilBert Model ===================
Accuracy: 0.44
=================== Albert Model ===================
Accuracy: 0.51
=================== BigBird Model ===================
Accuracy: 0.56
=================== Camembert Model ===================
Accuracy: 0.47


Παραρητώ πως το καλύτερο μοντέλο για το δεδομένο dataset είναι το BigBird Model. Καθώς χρησιμοποιώ αποκλειστικά προεκπαιδευμένα μοντέλα, χωρίς fine-tuning, όλα τα scores που προκύπτουν είναι ιδιαίτερα χαμηλά, δεδομένου ότι πρόκειται για δυαδική επιλογή.

## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["I love ice cream", "I like eating pizza"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.5788589

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
# insert your code here (load dataset)
dataset = load_dataset("truthful_qa","generation")
print(dataset)

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source'],
        num_rows: 817
    })
})


In [ ]:
truthful_dataset = dataset["validation"]
truthful_dataset = truthful_dataset.shuffle(seed = random.randint(1,100))
truthful_dataset = truthful_dataset.filter(lambda example: len(example["correct_answers"]) == 2).select([i for i in range(0,100)])

print(truthful_dataset)
print(truthful_dataset[0])

Filter:   0%|          | 0/817 [00:00<?, ? examples/s]

Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source'],
    num_rows: 100
})
{'type': 'Non-Adversarial', 'category': 'Misconceptions', 'question': 'What colors do dogs see?', 'best_answer': 'Dogs see yellow, blue, brown, and gray', 'correct_answers': ['Dogs see yellow, blue, brown, and gray', "A dog's color vision is similar to that of someone with red-green colorblindness"], 'incorrect_answers': ['Dogs see in black and white', 'Dogs cannot see color'], 'source': 'https://www.akc.org/expert-advice/health/are-dogs-color-blind/'}


In [ ]:
# insert your code here (load models for semantic similarity and QA)
from sentence_transformers import SentenceTransformer

model_mpnet_base_v2 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model_MiniLm_L6_v2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model_distilroberta_v1 = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
model_all_roberta_large_v1 = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
model_MiniLm_L12_v2 = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
model_distilbert_base_v4 = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-v4')


embeddings_1 = model_mpnet_base_v2.encode(sentences)
embeddings_2 = model_MiniLm_L6_v2.encode(sentences)
embeddings_3 = model_distilroberta_v1.encode(sentences)
embeddings_4 = model_all_roberta_large_v1.encode(sentences)
embeddings_5 = model_MiniLm_L12_v2.encode(sentences)
embeddings_6 = model_distilbert_base_v4.encode(sentences)

print(get_cosine_similarity(embeddings_1[0], embeddings_1[1]))
print(get_cosine_similarity(embeddings_2[0], embeddings_2[1]))
print(get_cosine_similarity(embeddings_3[0], embeddings_3[1]))
print(get_cosine_similarity(embeddings_4[0], embeddings_4[1]))
print(get_cosine_similarity(embeddings_5[0], embeddings_5[1]))
print(get_cosine_similarity(embeddings_6[0], embeddings_6[1]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.5788589
0.49340707
0.47997355
0.478817
0.44645905
0.29272088


In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)
import torch
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import numpy as np

def sentence_complete_4(model, tokenizer, sentence, best_answer, correct_answer_1, correct_answer_2, incorrect_answer):

  # Tokenize inputs
  inputs = tokenizer(
      [sentence] * 4,
      [best_answer, correct_answer_1, correct_answer_2, incorrect_answer],
      padding=True,
      truncation=True,
      return_tensors="pt"
  )

  input_ids = inputs["input_ids"].unsqueeze(0)
  attention_mask = inputs["attention_mask"].unsqueeze(0)

  # Forward pass
  model.eval()
  with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)

  logits = outputs.logits
  predicted_label = logits.argmax(dim=1)
  return predicted_label[0]



def compute_accuracy_4(model_multiple_choice, tokenizer_multiple_choice, model_semantic_similarity, threshold):
  correct_predictions = 0
  for i in range(0,100):
    question = truthful_dataset["question"][i]
    best_answer = truthful_dataset["best_answer"][i]
    correct_answer_1 = truthful_dataset["correct_answers"][i][0]
    correct_answer_2 = truthful_dataset["correct_answers"][i][1]
    incorrect_answer = truthful_dataset["incorrect_answers"][i][0]
    l = np.array(sentence_complete_4(model_multiple_choice, tokenizer_multiple_choice, question, best_answer, correct_answer_1, correct_answer_2, incorrect_answer))
    if l == 0: #predicted best answer
      correct_predictions = correct_predictions + 1
    elif l == 1 or l == 2:
      correct_answer = correct_answer_1 if l == 1 else correct_answer_2
      sentences = [best_answer, correct_answer]
      embeddings = model_semantic_similarity.encode(sentences)
      similarity = get_cosine_similarity(embeddings[0], embeddings[1])
      correct_predictions = (correct_predictions+1) if similarity >= threshold else correct_predictions

  accuracy = correct_predictions/100
  print("Accuracy: "+ str(accuracy))

In [ ]:
models_for_semantic_similarity=[model_mpnet_base_v2, model_MiniLm_L6_v2, model_distilroberta_v1, model_all_roberta_large_v1, model_MiniLm_L12_v2, model_distilbert_base_v4]
names_models_semantic_similarity=["Mpnet_V2", "MiniLm_L6_V2", "DistilRoberta_v1", "Roberta_Large_v1", "MiniLm_L12_v2", "Distilbert_v4"]

models_for_multiple_choice = [model_bert, model_distil, model_albert, model_bigbird, model_camem]
tokenizer_for_multiple_choice =[tokenizer_bert, tokenizer_distil, tokenizer_albert, tokenizer_bigbird, tokenizer_camem]
names_models_multiple_choice =["Bert", "DistilBert", "Albert", "BigBird", "CamemBird"]

for i in range(0,6):
  for j in range(0,5):
    print("Multiple choice model: "+names_models_multiple_choice[j]+"\nSentence transformer: "+names_models_semantic_similarity[i])
    compute_accuracy_4(models_for_multiple_choice[j], tokenizer_for_multiple_choice[j], models_for_semantic_similarity[i], threshold = 0.95)
    print("\n")


Multiple choice model: Bert
Sentence transformer: Mpnet_V2
Accuracy: 0.25


Multiple choice model: DistilBert
Sentence transformer: Mpnet_V2
Accuracy: 0.37


Multiple choice model: Albert
Sentence transformer: Mpnet_V2
Accuracy: 0.32


Multiple choice model: BigBird
Sentence transformer: Mpnet_V2
Accuracy: 0.29


Multiple choice model: CamemBird
Sentence transformer: Mpnet_V2
Accuracy: 0.38


Multiple choice model: Bert
Sentence transformer: MiniLm_L6_V2
Accuracy: 0.24


Multiple choice model: DistilBert
Sentence transformer: MiniLm_L6_V2
Accuracy: 0.36


Multiple choice model: Albert
Sentence transformer: MiniLm_L6_V2
Accuracy: 0.33


Multiple choice model: BigBird
Sentence transformer: MiniLm_L6_V2
Accuracy: 0.31


Multiple choice model: CamemBird
Sentence transformer: MiniLm_L6_V2
Accuracy: 0.37


Multiple choice model: Bert
Sentence transformer: DistilRoberta_v1
Accuracy: 0.24


Multiple choice model: DistilBert
Sentence transformer: DistilRoberta_v1
Accuracy: 0.36


Multiple choic

Για το συγκεκριμένο task θα χρησιμοποιήσω τα μοντέλα για multiple choice όπως πριν (sequence transformers), με την διαφορά πως τώρα είναι 4 οι πιθανές επιλογές. Για καθένα από αυτά θα δοκιμάσω έναν sentence transformer, ο οποίος θα χρησιμοποιηθεί για τον υπολογισμό των embeddings της προβλεπόμενης και της πραγματικής(best) λύσης. Αν η ομοιότητα συνημιτόνου μεταξύ αυτών είναι πάνω από το δεδομένο threshold, τότε θα θεωρήσω την απάντηση σωστή.

Ο καλύτερος συνδυασμός μοντέλων είναι CamemBird, MiniLm_L6_V2 και CamemBird, Roberta_Large_v1. Γενικά, και πάλι είναι πολύ χαμηλά τα σκορ (η τυχαία επιλογή θα έδινε ακρίβεια 0.25), για τον ίδιο λόγω με πριν.

Παρατηρώ ακόμη πως το μοντέλο για multiple choice CamemBird, που εδώ φάνηκε το καλύτερο, ήταν από τα χερότερα στο προηγούμενο task. Αυτό δείχνει πόσο ευαίσθητο είναι το μοντέλο στα διαφορετικά dataset όταν δεν γίνεται fine-tuning.

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [ ]:
# insert your code here (load dataset)
dataset = load_dataset("winogrande","winogrande_debiased")
print(dataset)

Generating train split:   0%|          | 0/9248 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 9248
    })
    test: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1767
    })
    validation: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1267
    })
})


In [ ]:
train_dataset = dataset["train"]

train_dataset = train_dataset.shuffle(seed=random. randint(10, 100))
train_dataset_1 = train_dataset.filter(lambda example: example["answer"] == "1").select([i for i in range(0, 50)])
train_dataset_2 = train_dataset.filter(lambda example: example["answer"] == "2").select([i for i in range(0, 50)])
train_dataset_ordered = concatenate_datasets([train_dataset_1, train_dataset_2])
wino_dataset = train_dataset_ordered.shuffle(seed=random. randint(10, 100))

print(wino_dataset)
print(wino_dataset[0])

Filter:   0%|          | 0/9248 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9248 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'option1', 'option2', 'answer'],
    num_rows: 100
})
{'sentence': "I measured the area in my kitchen, but the stove didn't fit because the _ was too small.", 'option1': 'stove', 'option2': 'area', 'answer': '2'}


Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [ ]:
# insert your code here (load models)
from transformers import AutoTokenizer, AutoModelForSequenceClassification

#transformer that was specifically fine-tuned for the winogrande dataset, i'm expecting better accuracy from this model
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/roberta-large-winogrande")
model_roberta_winogrande = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/roberta-large-winogrande")

model_name_bert = "bert-base-uncased"
model_name_distil = "distilbert-base-cased"
model_name_albert = "albert-base-v2"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
# insert your code here (create pipelines)
from transformers import pipeline

'''
text classification pipeline model not specifically trained on Winogrande, trained for Natural Language Inference
the classes are:
Entailment: The premise entails the hypothesis.
Contradiction: The premise contradicts the hypothesis.
Neutral: The premise neither entails nor contradicts the hypothesis.
'''
# Create a pipeline for sequence classification
nlp_bert = pipeline("text-classification", model = model_name_bert) #Specifying "text-classification" ensures that the pipeline configures the underlying model for the classification task
nlp_distil = pipeline("text-classification", model = model_name_distil)
nlp_albert = pipeline("text-classification", model = model_name_albert)

'''
model specifically fine-tuned for the winogrande dataset, classes are 1, 2 for the 2 options
'''
nlp_roberta = pipeline("text-classification", model = "DeepPavlov/roberta-large-winogrande")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# insert your code here (function for predicting best fill)

'''
Decision Logic:
If both options receive the same label and the label is not contradiction, the decision is based on the scores, if both labels are contradiction, the decision is based inversely on the scores.
If only option2 is labeled as contradiction, and if option1 (answer '1') is correct per the dataset, it is counted as correct.
Similarly, if option1 is labeled as contradiction and the correct answer is option2, it counts as correct.
If option1 is labeled as neutral and option2 has a stronger label like entailment, and the answer is '2', it counts as correct.
If option2 is labeled as neutral and option1 has a stronger label, and the answer is '1', it counts as correct.
'''
def accuracy_of_model_nli(model,dataset):
  correct_answers=0.0
  for row in wino_dataset:
    sentence=row['sentence']
    option1=row['option1']
    option2=row['option2']
    answer=row['answer']
    outputs=model([sentence+option1,sentence+option2])
    label1=outputs[0]['label']
    label2=outputs[1]['label']
    score1=outputs[0]['score'] #the confidence score for the label assigned by the model, indicating how confident the model is that the label is correct
    score2=outputs[1]['score']
    if(label1==label2):
      if(label1!='contradiction' and label1!='CONTRADICTION'):
        if((score1>score2 and answer=='1') or (score1<score2 and answer=='2')):
          correct_answers=correct_answers+1
      elif((score1<score2 and answer=='1') or (score1>score2 and answer=='2')):
          correct_answers=correct_answers+1
    elif(label2=='contradiction' or label2=='CONTRADICTION'):  #the other is entailment or neutral=>better
      if(answer=='1'):correct_answers=correct_answers+1
    elif(label1=='contradiction' or label1=='CONTRADICTION'):
      if(answer=='2'):correct_answers=correct_answers+1
    elif(label1=='neutral' or label1=='NEUTRAL'):    #the other is entailment=>better
      if(answer=='2'):correct_answers=correct_answers+1
    elif(label2=='neutral' or label2=='NEUTRAL'):
      if(answer=='1'):correct_answers=correct_answers+1
  return correct_answers/100

In [ ]:
print("=================== Bert Model ===================")
print(accuracy_of_model_nli(nlp_bert,train_dataset))
print("=================== DistilBert Model ===================")
print(accuracy_of_model_nli(nlp_distil,train_dataset))
print("=================== Albert Model ===================")
print(accuracy_of_model_nli(nlp_albert,train_dataset))

=================== Bert Model ===================
0.54
=================== DistilBert Model ===================
0.44
=================== Albert Model ===================
0.37


In [ ]:
def accuracy_of_model_roberta(model,dataset):
  correct_answers=0.0
  for row in wino_dataset:
    sentence=row['sentence']
    option1=row['option1']
    option2=row['option2']
    answer=row['answer']
    outputs=model([sentence+option1,sentence+option2])
    label1=outputs[0]['label'] #The prediction of the model, indicating whether the sentence completion is true or false in the context
    label2=outputs[1]['label']
    score1=outputs[0]['score']
    score2=outputs[1]['score']
    if(label1==label2):
      if(label1=='True'):
        if((score1>score2 and answer=='1') or (score1<score2 and answer=='2')):
          correct_answers=correct_answers+1
      elif((score1<score2 and answer=='1') or (score1>score2 and answer=='2')):
          correct_answers=correct_answers+1
    elif(label2=='False'):  #the other is true
      if(answer=='1'):correct_answers=correct_answers+1
    elif(label1=='False'):
      if(answer=='2'):correct_answers=correct_answers+1
  return correct_answers/100

In [ ]:
print("=================== Roberta Model fine-tuned for the Winongrande dataset ===================")
accuracy_of_model_roberta(nlp_roberta,train_dataset)

=================== Roberta Model fine-tuned for the Winongrande dataset ===================


0.99

Αυτό το task ανάγεται στο Natural Language Inference, οπότε θα χρησιμοποιήσω 3 transformers εκπαιδευμένους σε αυτό. Τα μοντέλα αυτά παίρνουν ως είσοδο ένα batch με τις δύο πιθανές προτάσεις και δίνουν ως έξοδο για καθεμία από αυτές την κλάση (contradiction, neutral, entailment) και το σκορ εμπιστοσύνης για την επιλογή αυτή. Συγκρίνοντας τα αποτελέσματα, αποφασίζω ποια πρόταση είναι πιο σωστή, δηλαδή ποια διαλέγει το μοντέλο μου, και ελέγχω αν είναι η σωστή απάντηση.

Τα μοντέλα αυτά δεν δίνουν πολύ καλά σκορ ακρίβειας, καθώς το task στο οποίο είναι εκπαιδευμένα δεν είναι πολύ σχετικό με το δικό μας, απλά περιέχουν και τα δύο το κομμάτι της νοηματικής κατανόησης του κειμένου.

Χρησιμοποίησα κι ένα μοντέλο εκπαιδευμένο στο winogrande dataset, το οποίο έδωσε τέλεια ακρίβεια.